# 6-Calibration: MELTS liquid model with ZrSiO<sub>4</sub>

**Parameter and calibration exploration notebook.**  
**Run Endmembers-MELTS, Liquid-MELTS-codegen, Liquid-MELTS-API-tests and Liquid=MELTS-calib-1 notebooks first!**  

Proposed associated solution model:

| Components: | Species: | Reaction: | Variable: | Component: | Species X: |
| :---------- | :------- | :-------- | :-------- | :--------- | :--------- |
| SiO<sub>2</sub> | SiO<sub>2</sub> | | | $n_1$ | $X_1$ |
| TiO<sub>2</sub> | TiO<sub>2</sub> | | $r_1$ | $n_2$ | $X_2$ |
| Al<sub>2</sub>O<sub>3</sub> | Al<sub>2</sub>O<sub>3</sub> | | $r_2$ | $n_3$ | $X_3$ |
| Fe<sub>2</sub>O<sub>3</sub> | Fe<sub>2</sub>O<sub>3</sub> | | $r_3$ | $n_4$ | $X_4$ |
| MgCr<sub>2</sub>O<sub>4</sub> | MgCr<sub>2</sub>O<sub>4</sub> | | $r_4$ | $n_5$ | $X_5$ |
| Fe<sub>2</sub>SiO<sub>4</sub> | Fe<sub>2</sub>SiO<sub>4</sub> | | $r_5$ | $n_6$ | $X_6$ |
| MnSi<sub>1/2</sub>O<sub>2</sub> | MnSi<sub>1/2</sub>O<sub>2</sub> | | $r_6$ | $n_7$ | $X_7$ |
| Mg<sub>2</sub>SiO<sub>4</sub> | Mg<sub>2</sub>SiO<sub>4</sub> | | $r_7$ | $n_8$ | $X_8$ |
| NiSi<sub>1/2</sub>O<sub>2</sub> | NiSi<sub>1/2</sub>O<sub>2</sub>  | | $r_8$ | $n_9$ | $X_9$ |
| CoSi<sub>1/2</sub>O<sub>2</sub> | CoSi<sub>1/2</sub>O<sub>2</sub> | | $r_9$ | $n_{10}$ | $X_{10}$ |
| CaSiO<sub>3</sub> | CaSiO<sub>3</sub> | | $r_{10}$ | $n_{11}$ | $X_{11}$ |
| Na<sub>2</sub>SiO<sub>3</sub> | Na<sub>2</sub>SiO<sub>3</sub> | | $r_{11}$ | $n_{12}$ | $X_{12}$ |
| KAlSiO<sub>4</sub> | KAlSiO<sub>4</sub> | | $r_{12}$ | $n_{13}$ | $X_{13}$ |
| Ca<sub>3</sub>(PO<sub>4</sub>)<sub>2</sub> | Ca<sub>3</sub>(PO<sub>4</sub>)<sub>2</sub> | | $r_{13}$ | $n_{14}$ | $X_{14}$ |
| H<sub>2</sub>O | H<sub>2</sub>O | | $r_{14}$ | $n_{15}$ | $X_{15}$ |
| CO<sub>2</sub> | CO<sub>2</sub> | | $r_{15}$ | $n_{16}$ | $X_{16}$ |
| ZrSiO<sub>4</sub> | ZrSiO<sub>4</sub> | | $r_{16}$ | $n_{17}$ | $X_{17}$ |
| | CaCO<sub>3</sub> | CO<sub>2</sub> + CaSiO<sub>3</sub> = CaCO<sub>3</sub> + SiO<sub>2</sub> | $s_1$ | | $X_{18}$ |
| | Na<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> | ZrSiO<sub>4</sub> + 2Na<sub>2</sub>SiO<sub>3</sub> = Na<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> + SiO<sub>2</sub> | $s_2$ | | $X_{19}$ |
| | K<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> | ZrSiO<sub>4</sub> + 4KAlSiO<sub>4</sub> = K<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> + 2Al<sub>2</sub>O<sub>3</sub> + 3SiO<sub>2</sub> | $s_3$ | | $X_{20}$ |

In [ ]:
import pandas as pd
import numpy as np
from thermoengine import core, coder, phases
import sympy as sym
import math
sym.init_printing()
from thermoengine.model import Database
db = Database()
import matplotlib.pyplot as plt
import scipy as sci
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

- Number of components = 17
- Number of species = 20
- Number of "ordering parameters" (i.e., dependent species) = 3

In [ ]:
nc = 17
nw = 20
ns = 3

Import previously generated and compiled model

In [ ]:
model_working_dir = "working"
%cd {model_working_dir}
from pyximport import install
install()
import rMELTS_ZR
%cd ..

Test identifier routines

In [ ]:
print(rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_identifier())
print(rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_name())

## Read in parameter estimates from first calibration notebook

In [ ]:
import pickle
with open('params.pickle', 'rb') as f:
    params_stored = pickle.load(f)
for i,x in enumerate(params_stored):
    rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(i, x) 

## Create models for zirconium-bearing minerals

In [ ]:
model = coder.StdStateModel()
model.set_module_name('zirconium_minerals')
model_type = "calib"
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

### Formulation, Robie RA, Hemingway BS, 1995
Thermodynamic Properties of Minerals and Related Substances at 298.15 K and 1 Bar (10<sup>5</sup> Pascals) Pressure and at Higher Temperatures. U.S. Geological Survey, Bulletin 2131, 461pp  

- $\Delta H_f$, S, V, Cp
- $Cp = A_1 + A_2 T + \frac{A_3}{T^2} + \frac{A_4}{\sqrt{T}} + A_5 T^2$

In [ ]:
h0,s0,a1,a2,a3,a4,a5,v = sym.symbols('h0 s0 a1 a2 a3 a4 a5 v')
params = [
    ('h0','J',h0), 
    ('s0','J/K', s0), 
    ('a1', 'J/K', a1), 
    ('a2', 'J/K^2', a2), 
    ('a3', 'J-K', a3), 
    ('a4', 'J/K^1/2', a4), 
    ('a5', 'J/K^3', a5),
    ('v', 'J/bar', v)
]
Cp = a1 + a2*T + a3/T**2 + a4/sym.sqrt(T) + a5*T**2
G = h0 + sym.integrate(Cp,(T,Tr,T)) - T*(s0 + sym.integrate(Cp/T,(T,Tr,T))) + sym.integrate(v,(P,Pr,P))
model.add_expression_to_model(G, params)

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
param_dict = {
    'Phase': 'zircon',
    'Formula': 'Zr(1)Si(1)O(4)',
    'h0':  -2034200.0,
    's0':        84.0,
    'v':          3.926,
    'a1':         2.370e2,
    'a2':        -1.788e-2,
    'a3':        -1.496e5,
    'a4':        -2.268e3,
    'a5':         0.0,
    'T_r':      298.15,
    'P_r':        1.0,
    'trl':     1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cp zirconium_minerals.pyx endmembers.pyx
endmember_file_list = ['zircon_zirconium_minerals_calib.c']
endmember_name_list = ['zircon']

In [ ]:
param_dict = {
    'Phase': 'baddeleyite',
    'Formula': 'Zr(1)O(2)',
    'h0':  -1100600.0,
    's0':        50.4,
    'a1':         1.073e2,
    'a2':        -5.011e-3,
    'a3':        -2.203e5,
    'a4':        -8.141e2,
    'a5':         0.0,
    'v':          2.115,
    'T_r':      298.15,
    'P_r':        1.0,
    'trl':     1673.15
}
result = model.create_code_module(phase=param_dict['Phase'], formula=param_dict['Formula'], params=param_dict, module_type=model_type)
%cat zirconium_minerals.pyx >> endmembers.pyx
endmember_file_list.append('baddeleyite_zirconium_minerals_calib.c')
endmember_name_list.append('baddeleyite')

In [ ]:
with open('zirconium_minerals.pyxbld', 'r') as f:
    fold = f.read()
    f.close()
old_file_list = "'" + endmember_file_list[-1] + "'"
new_file_list = ''
sep = ''
for x in endmember_file_list:
    new_file_list += sep + "'" + x + "'"
    sep = ", "
fnew = fold.replace(old_file_list, new_file_list)
with open('zirconium_minerals.pyxbld', 'w') as f:
    f.write(fnew)
    f.close
%cp endmembers.pyx zirconium_minerals.pyx

In [ ]:
import zirconium_minerals
%cd ..

## Read common scripts
Execute scripts in the Jupyter notebook namespace so that methods have access to notebook variables.

In [ ]:
%run -i support_scripts.py

## Function to calculate affinities for these reactions:
ZrSiO<sub>4</sub> = ZrSiO<sub>4</sub> (zircon)  
ZrSiO<sub>4</sub> = ZrO<sub>2</sub> (baddeleyite)  + SiO<sub>2</sub>

In [ ]:
def zirconium_min_affinities(t, p, mu, print_results=True):
    global zirconium_minerals
    G0zirc = zirconium_minerals.cy_zircon_zirconium_minerals_calib_g(t,p)
    G0badd = zirconium_minerals.cy_baddeleyite_zirconium_minerals_calib_g(t,p)
    affinities = []
    affinities.append(mu[16] - G0zirc)
    affinities.append(mu[16] - G0badd - mu[0])
    if print_results:
        print ('Affinity {0:<12.12s} {1:12.3f}'.format('zircon', affinities[0]))
        print ('Affinity {0:<12.12s} {1:12.3f}'.format('baddeleyite', affinities[1]))
    return tuple(affinities)

## Guess some parameter values (from a first pass through the primary calibration data)
- Exchange entropies and volumes are maintained at zero
- Free parameters are enthalpies of ZrSiO<sub>4</sub>, Na<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> and K<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub>, and the entropy and volume of ZrSiO<sub>4</sub>  

These estimates are arrived at by guess work minimization of residuals.  They are not final values.

In [ ]:
ref_param_values = rMELTS_ZR.cy_Liquid_rMELTS_ZR_get_param_values()
ref_param_values

In [ ]:
adj_H_ZrSiO4 = 100000.0 + 22020.0    + 107500.0    - 32110.0
adj_S_ZrSiO4 =               40.0242 +     83.7465 -    27.8937
adj_V_ZrSiO4 =                0.5448 -      0.2236 -     0.110
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(5,  ref_param_values[ 5] + adj_H_ZrSiO4 - 225000.0) # Na4ZrSi2O8
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(6,  ref_param_values[ 6] + adj_S_ZrSiO4)
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(7,  ref_param_values[ 7] + adj_V_ZrSiO4)
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(8,  ref_param_values[ 8] + adj_H_ZrSiO4 -  75000.0) # K4ZrSi2O8
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(9,  ref_param_values[ 9] + adj_S_ZrSiO4)
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(10, ref_param_values[10] + adj_V_ZrSiO4)
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(11, ref_param_values[11] + adj_H_ZrSiO4)            # ZrSiO4
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(12, ref_param_values[12] + adj_S_ZrSiO4)
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(13, ref_param_values[13] + adj_V_ZrSiO4)

In [ ]:
param_values = rMELTS_ZR.cy_Liquid_rMELTS_ZR_get_param_values()
param_values

## Evaluate zircon saturation data
- two primary sources:
  - Watson EB and Harrison TM (1983) Zircon saturation revisited: temperature and composition effects in a variety of crustal magma types. Earth and Planetary Science Letters, 64, 295-304
  - Boehnke P, Watson EB, Trail D, Harrison TM, Schmitt AK (2013) Zircon saturation re-visited. Chemical Geology, 351, 324-334

Read data sources (Excel files) as Pandas dataframes

In [ ]:
df1 = pd.read_csv('Watson_and_Harrison.csv')
df1.fillna(0)

In [ ]:
for i,x in enumerate(df1.columns):
    print (i+1, x)

In [ ]:
df2 = pd.read_csv('Boehnke_et_al.csv')
df2.fillna(0)

In [ ]:
for i,x in enumerate(df2.columns):
    print (i+1, x)

Process the two data sets and store residual information and other results in the following Python lists:
- x_data[] - data index 
- y_data[] - residual
- n_liq[] - liquid compositional vector
- s_liq[] - liquid species concentration vector
- t_data[] - temperature in Kelvins
- p_data[] - pressure in bars
- name[] - source string or experimental descriptor
- s_data[] - source author id (1, 2)
- y_est[] - extimated value of Zr liquid concentration in PPM (value that zeros residuals)
- y_con[] - True or False regarding convergence of y_est
- y_ref[] - measured value of Zr liquid concentration

In [ ]:
name = []
x_data = []
y_data = []
n_liq = []
s_liq = []
t_data = []
p_data = []
s_data = []
y_est = []
y_con = []
y_ref = []
auth_dat = []
for df in [df1, df2]:
    df_iterator = df.itertuples()
    #
    for row_indx,data in enumerate(df_iterator):
        auth_id = data[3]
        t = data[25 if auth_id == 1 else 17] + 273.15
        p = data[26 if auth_id == 1 else 18]*10000.0
        if data[4] != 'Liquid':
            continue
        grm_oxides = {
            'SiO2':  data[5 if auth_id == 1 else 5], 
            'TiO2':  data[7 if auth_id == 1 else 6], 
            'Al2O3': data[9 if auth_id == 1 else 7], 
            'Fe2O3': 0.0,
            'Cr2O3': 0.0, 
            'FeO':   data[11 if auth_id == 1 else 8], 
            'MnO':    0.0,
            'MgO':   data[13 if auth_id == 1 else 9], 
            'NiO':    0.0, 
            'CoO':    0.0,
            'CaO':   data[15 if auth_id == 1 else 10], 
            'Na2O':  data[17 if auth_id == 1 else 11], 
            'K2O':   data[19 if auth_id == 1 else 12], 
            'P2O5':   0.0, 
            'H2O':   data[24 if auth_id == 1 else 15],
            'CO2':    0.01
        }
        tot_grm_oxides = 0.0
        for key in grm_oxides.keys():
            tot_grm_oxides += grm_oxides[key]
        mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
        #
        NNO = db.redox_buffer(t, p, buffer='NNO', method='LEPR')
        ox_moles = mol_oxides.reshape(1,nc-1)
        db.redox_state(t, p, 
                       oxide_comp={'Liq':ox_moles}, 
                       logfO2=np.array(NNO), 
                       phase_of_interest='Liq', method='Kress91')
        #print ('n Fe2O3   at NNO   moles = {0:10.8f}'.format(ox_moles[0,3]))
        #print ('n FeO     at NNO   moles = {0:10.8f}'.format(ox_moles[0,5]))
        mol_oxides[3] = ox_moles[0,3]
        mol_oxides[5] = ox_moles[0,5]
        #
        mol_elm = core.chem.mol_oxide_to_elem(mol_oxides)
        elm = np.zeros(107)
        for i,sym in enumerate(
            ['Si', 'Ti', 'Al', 'Fe', 'Cr', 'Mn', 'Mg', 'Ni', 'Co', 'Ca', 'Na', 'K', 'P', 'H', 'C', 'O']):
            index = core.chem.PERIODIC_ORDER.tolist().index(sym)
            elm[index] = mol_elm[i]
        #
        wtZrO2 = data[22 if auth_id == 1 else 13]
        if wtZrO2 > 1.0:
            continue
        auth_dat.append(auth_id)
        y_ref.append(wtZrO2)
        elm_ref = np.copy(elm)
        elm[core.chem.PERIODIC_ORDER.tolist().index('Zr')] += wtZrO2/123.218
        elm[core.chem.PERIODIC_ORDER.tolist().index('O')]  += wtZrO2*2.0/123.218
        n = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_conv_elm_to_moles(elm)
        if not rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_test_moles(n):
            print ('Output of intrinsic composition calculation fails tests for permissible values.')
            continue
        s = rMELTS_ZR.cy_Liquid_rMELTS_ZR_order_params(t, p, n)
        mu = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_dgdn(t,p,n)
        y = zirconium_min_affinities(t, p, mu, print_results=False)[0]
        #
        # Estimate zirconium content by zeroing the residuals
        def residual(log_wtZrO2_est, doprint=False):
            global elm_ref, t, p
            wtZrO2_est = np.exp(log_wtZrO2_est)
            elm_est = np.copy(elm_ref)
            elm_est[core.chem.PERIODIC_ORDER.tolist().index('Zr')] += wtZrO2_est/123.218
            elm_est[core.chem.PERIODIC_ORDER.tolist().index('O')]  += wtZrO2_est*2.0/123.218
            n_est = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_conv_elm_to_moles(elm_est)
            mu_est= rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_dgdn(t,p,n_est)
            result = zirconium_min_affinities(t, p, mu_est, print_results=False)[0]
            if doprint:
                print (wtZrO2_est, result)
            return result
        #
        try:
            sol = sci.optimize.root_scalar(residual, 
                                           #x0=np.log(wtZrO2), 
                                           #x1=np.log(wtZrO2*10.), 
                                           rtol=1.e-5,
                                           bracket=[-5,2.3],
                                           method='brentq',
                                           maxiter=50)
            print (sol)
            y_est.append(np.exp(sol.root))
            y_con.append(sol.converged)
        except ValueError:
            print ('Solution not within brackets')
            y_est.append(0.0)
            y_con.append(False)
        #
        print (auth_id, data[2], y)
        s_data.append(auth_id)
        name.append(data[2])
        x_data.append(row_indx)
        y_data.append(y)
        n_liq.append(n)
        s_liq.append(s)
        t_data.append(t)
        p_data.append(p)

Residual statistics and a linear regression that provides rough estimates of H, S and V forZrSiO<sub>4</sub>

In [ ]:
print ('Average residual:', np.mean(np.array(y_data)))
print ('Std Dev residual:', np.std(np.array(y_data)))

In [ ]:
import statsmodels.api as sm

In [ ]:
X = sm.add_constant(np.column_stack((t_data, p_data)))
results = sm.OLS(np.array(y_data), X).fit()
results.summary()

### Analysis of residuals

Back calculation of Zr concentrations made by zeroing residuals

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(np.array([x for i,x in enumerate(y_ref) if y_con[i]]), 
         np.array([x for i,x in enumerate(y_est) if y_con[i]]), 'r+')
plt.plot(np.array([x for i,x in enumerate(y_ref) if not y_con[i]]), 
         np.array([x for i,x in enumerate(y_est) if not y_con[i]]), 'b+')
plt.plot(np.array([0,1]), np.array([0,1]), 'g-')
plt.ylabel('wt ZrO2 est')
plt.xlabel('wt ZrO2 data')
plt.subplot(1,2,2)
plt.plot(np.array([91.224*1000000.0*x/100.0/123.218 for i,x in enumerate(y_ref) if y_con[i]]), 
         np.array([91.224*1000000.0*x/100.0/123.218 for i,x in enumerate(y_est) if y_con[i]]), 'r+')
plt.plot(np.array([0,7000]), np.array([0,7000]), 'g-')
plt.ylabel('ppm ZrO2 est')
plt.xlabel('ppm ZrO2 data')
#axes = plt.gca()
#axes.set_xlim(0,1)
plt.tight_layout()
plt.show

Residuals from fitting the two data sets (red is Watson and Harrison, blue is Boehnke et al.)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.plot([min(x_data),max(x_data)], [0.0,0.0], 'g-')
plt.plot(np.array([x_data[i] for i,x in enumerate(s_data) if x == 1]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 1]), 'r+')
plt.plot(np.array([x_data[i] for i,x in enumerate(s_data) if x == 2]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 2]), 'b+')
plt.ylabel('residual')
plt.xlabel('index')
plt.subplot(1,3,2)
plt.plot([min(np.array(n_liq)[:,0]),max(np.array(n_liq)[:,0])], [0.0,0.0], 'g-')
plt.plot(np.array([np.array(n_liq)[i,0] for i,x in enumerate(s_data) if x == 1]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 1]), 'r+')
plt.plot(np.array([np.array(n_liq)[i,0] for i,x in enumerate(s_data) if x == 2]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 2]), 'b+')
plt.ylabel('residual')
plt.xlabel('X SiO2')
plt.subplot(1,3,3)
plt.plot([min(np.array(n_liq)[:,16]),max(np.array(n_liq)[:,16])], [0.0,0.0], 'g-')
plt.plot(np.array([np.array(n_liq)[i,16] for i,x in enumerate(s_data) if x == 1]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 1]), 'r+')
plt.plot(np.array([np.array(n_liq)[i,16] for i,x in enumerate(s_data) if x == 2]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 2]), 'b+')
plt.ylabel('residual')
plt.xlabel('X ZrSiO4')
plt.tight_layout()
plt.show()

Examine residual (zircon affinity) dependence on alkali and water concentrations

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.plot([min(np.array(n_liq)[:,11]),max(np.array(n_liq)[:,11])], [0.0,0.0], 'g-')
plt.plot(np.array([np.array(n_liq)[i,11] for i,x in enumerate(s_data) if x == 1]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 1]), 'r+')
plt.plot(np.array([np.array(n_liq)[i,11] for i,x in enumerate(s_data) if x == 2]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 2]), 'b+')
plt.ylabel('residual')
plt.xlabel('X Na2SiO3')
plt.subplot(1,3,2)
plt.plot([min(np.array(n_liq)[:,12]),max(np.array(n_liq)[:,12])], [0.0,0.0], 'g-')
plt.plot(np.array([np.array(n_liq)[i,12] for i,x in enumerate(s_data) if x == 1]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 1]), 'r+')
plt.plot(np.array([np.array(n_liq)[i,12] for i,x in enumerate(s_data) if x == 2]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 2]), 'b+')
plt.ylabel('residual')
plt.xlabel('X KAlSiO4')
plt.subplot(1,3,3)
plt.plot([min(np.array(n_liq)[:,14]),max(np.array(n_liq)[:,14])], [0.0,0.0], 'g-')
plt.plot(np.array([np.array(n_liq)[i,14] for i,x in enumerate(s_data) if x == 1]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 1]), 'r+')
plt.plot(np.array([np.array(n_liq)[i,14] for i,x in enumerate(s_data) if x == 2]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 2]), 'b+')
plt.ylabel('residual')
plt.xlabel('X H2O')
plt.tight_layout()
plt.show()

Examine residual (zircon affinity) dependence on temperature

In [ ]:
plt.plot([min(t_data),max(t_data)], [0.0,0.0], 'g-')
plt.plot(np.array([t_data[i] for i,x in enumerate(s_data) if x == 1]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 1]), 'r+')
plt.plot(np.array([t_data[i] for i,x in enumerate(s_data) if x == 2]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 2]), 'b+')
plt.ylabel('residual')
plt.xlabel('T K')
plt.show()

Examine residual (zircon affinity) dependence on pressure

In [ ]:
plt.plot([min(p_data),max(p_data)], [0.0,0.0], 'g-')
plt.plot(np.array([p_data[i] for i,x in enumerate(s_data) if x == 1]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 1]), 'r+')
plt.plot(np.array([p_data[i] for i,x in enumerate(s_data) if x == 2]),
         np.array([y_data[i] for i,x in enumerate(s_data) if x == 2]), 'b+')

plt.ylabel('residual')
plt.xlabel('P (GPa)')
plt.show()

Examine Zr-speciation of the liquid

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.plot(np.array(x_data), np.array(n_liq)[:,16], 'r+')
plt.ylabel('n ZrSiO4')
plt.xlabel('index')
plt.subplot(1,3,2)
plt.plot(np.array(x_data), np.array(s_liq)[:,1], 'r+')
plt.ylabel('n Na4ZrSi2O8')
plt.xlabel('index')
plt.subplot(1,3,3)
plt.plot(np.array(x_data), np.array(s_liq)[:,2], 'r+')
plt.ylabel('n K4ZrSi2O8')
plt.xlabel('index')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.semilogy(np.array(x_data), np.array(n_liq)[:,16], 'r+', label='ZrSiO4')
plt.semilogy(np.array(x_data), np.array(s_liq)[:,1],  'g+', label='Na4ZrSi2O8')
plt.semilogy(np.array(x_data), np.array(s_liq)[:,2],  'b+', label='K4ZrSi2O8')
plt.ylabel('n Zr species')
plt.xlabel('index')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.semilogy(np.array(n_liq)[:,11], np.array(n_liq)[:,16], 'r+', label='ZrSiO4')
plt.semilogy(np.array(n_liq)[:,11], np.array(s_liq)[:,1],  'g+', label='Na4ZrSi2O8')
plt.semilogy(np.array(n_liq)[:,11], np.array(s_liq)[:,2],  'b+', label='K4ZrSi2O8')
plt.ylabel('n Zr species')
plt.xlabel('n Na2SiO3')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.semilogy(np.array(n_liq)[:,12], np.array(n_liq)[:,16], 'r+', label='ZrSiO4')
plt.semilogy(np.array(n_liq)[:,12], np.array(s_liq)[:,1],  'g+', label='Na4ZrSi2O8')
plt.semilogy(np.array(n_liq)[:,12], np.array(s_liq)[:,2],  'b+', label='K4ZrSi2O8')
plt.ylabel('n Zr species')
plt.xlabel('n KAlSiO4')
plt.legend()
plt.show()

Remember these results before lists are over written with the peralkaline dataset

In [ ]:
name_all   = [] + name
x_data_all = [] + x_data 
y_data_all = [] + y_data
n_liq_all  = [] + n_liq
s_liq_all  = [] + s_liq
t_data_all = [] + t_data
p_data_all = [] + p_data
s_data_all = [] + s_data
y_est_all  = [] + y_est
y_con_all  = [] + y_con
y_ref_all  = [] + y_ref

## Evaluate peralkaline compositions reported in Watson (1979)

In [ ]:
df3 = pd.read_excel('Watson_1979.xlsx')
df3.fillna(0)

In [ ]:
for i,x in enumerate(df3.columns):
    print (i+1, x)

Re-estimate "optimal" parameters for this data set 

In [ ]:
print(ref_param_values)

In [ ]:
adj_H_ZrSiO4 = 100000.0 + 22020.0    + 107500.0    - 32110.0
adj_S_ZrSiO4 =               40.0242 +     83.7465 -    27.8937
adj_V_ZrSiO4 =                0.5448 -      0.2236 -     0.110
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(5,  ref_param_values[ 5] + adj_H_ZrSiO4 - 225000.0 - 50000.0) # Na4ZrSi2O8
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(6,  ref_param_values[ 6] + adj_S_ZrSiO4)
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(7,  ref_param_values[ 7] + adj_V_ZrSiO4)
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(8,  ref_param_values[ 8] + adj_H_ZrSiO4 -  75000.0 - 50000.0)  # K4ZrSi2O8
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(9,  ref_param_values[ 9] + adj_S_ZrSiO4)
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(10, ref_param_values[10] + adj_V_ZrSiO4)
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(11, ref_param_values[11] + adj_H_ZrSiO4) # ZrSiO4
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(12, ref_param_values[12] + adj_S_ZrSiO4)
rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(13, ref_param_values[13] + adj_V_ZrSiO4)

In [ ]:
param_values = rMELTS_ZR.cy_Liquid_rMELTS_ZR_get_param_values()
print (param_values)

Analyze the data set

In [ ]:
df_iterator = df3.itertuples()
name = []
x_data = []
y_data = []
n_liq = []
s_liq = []
t_data = []
p_data = []
s_data = []
y_est = []
y_con = []
y_ref = []
jacobian = []
for row_indx,data in enumerate(df_iterator):
    auth_id = data[1]
    t = data[4] + 273.15
    p = data[5]
    grm_oxides = {
        'SiO2':  data[6], 
        'TiO2':  0.0, 
        'Al2O3': data[7], 
        'Fe2O3': max(data[8], 0.01),
        'Cr2O3': 0.0, 
        'FeO':   0.0, 
        'MnO':   0.0,
        'MgO':   0.0, 
        'NiO':   0.0, 
        'CoO':   0.0,
        'CaO':   max(data[9], 0.01), 
        'Na2O':  data[10], 
        'K2O':   data[11], 
        'P2O5':  0.0, 
        'H2O':   6.5,
        'CO2':   0.01
    }
    auth_dat.append(3)
    tot_grm_oxides = 0.0
    for key in grm_oxides.keys():
        tot_grm_oxides += grm_oxides[key]
    mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
    #
    if data[8] > 0.0:
        NNO = db.redox_buffer(t, p, buffer='NNO', method='LEPR')
        ox_moles = mol_oxides.reshape(1,nc-1)
        db.redox_state(t, p, 
                       oxide_comp={'Liq':ox_moles}, 
                       logfO2=np.array(NNO), 
                       phase_of_interest='Liq', method='Kress91')
        #print ('n Fe2O3   at NNO   moles = {0:10.8f}'.format(ox_moles[0,3]))
        #print ('n FeO     at NNO   moles = {0:10.8f}'.format(ox_moles[0,5]))
        mol_oxides[3] = ox_moles[0,3]
        mol_oxides[5] = ox_moles[0,5]
        #
    mol_elm = core.chem.mol_oxide_to_elem(mol_oxides)
    elm = np.zeros(107)
    for i,sym in enumerate(
        ['Si', 'Ti', 'Al', 'Fe', 'Cr', 'Mn', 'Mg', 'Ni', 'Co', 'Ca', 'Na', 'K', 'P', 'H', 'C', 'O']):
        index = core.chem.PERIODIC_ORDER.tolist().index(sym)
        elm[index] = mol_elm[i]
    #
    wtZrO2 = 100.0*123.218*data[2]/91.224/1000000.0
    y_ref.append(wtZrO2)
    elm_ref = np.copy(elm)
    elm[core.chem.PERIODIC_ORDER.tolist().index('Zr')] += wtZrO2/123.218
    elm[core.chem.PERIODIC_ORDER.tolist().index('O')]  += wtZrO2*2.0/123.218
    n = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_conv_elm_to_moles(elm)
    if not rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_test_moles(n):
        print ('Output of intrinsic composition calculation fails tests for permissible values.')
        continue
    s = rMELTS_ZR.cy_Liquid_rMELTS_ZR_order_params(t, p, n)
    mu = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_dgdn(t, p, n)
    y = zirconium_min_affinities(t, p, mu, print_results=False)[0]
    #
    # Estimate zirconium content by zeroing the residuals
    def residual(log_wtZrO2_est, doprint=False):
        global elm_ref, t, p
        wtZrO2_est = np.exp(log_wtZrO2_est)
        elm_est = np.copy(elm_ref)
        elm_est[core.chem.PERIODIC_ORDER.tolist().index('Zr')] += wtZrO2_est/123.218
        elm_est[core.chem.PERIODIC_ORDER.tolist().index('O')]  += wtZrO2_est*2.0/123.218
        n_est = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_conv_elm_to_moles(elm_est)
        mu_est= rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_dgdn(t,p,n_est)
        result = zirconium_min_affinities(t, p, mu_est, print_results=False)[0]
        if doprint:
            print (wtZrO2_est, result)
        return result
    #
    try:
        sol = sci.optimize.root_scalar(residual, 
                                       #x0=np.log(wtZrO2), 
                                       #x1=np.log(wtZrO2*10.), 
                                       rtol=1.e-5,
                                       bracket=[-5,2.3],
                                       method='brentq',
                                       maxiter=50)
        print (sol)
        y_est.append(np.exp(sol.root))
        y_con.append(sol.converged)
    except ValueError:
        print ('Solution not within brackets')
        y_est.append(0.0)
        y_con.append(False)
    #
    dparam = []
    for i in range(0,rMELTS_ZR.cy_Liquid_rMELTS_ZR_get_param_number()):
        D2gDnDparam = rMELTS_ZR.cy_Liquid_rMELTS_ZR_dparam_dgdn(t, p, n, i)
        dparam.append(D2gDnDparam[16])
    #
    print (auth_id, data[2], y)
    s_data.append(auth_id)
    name.append(data[12])
    x_data.append(row_indx)
    y_data.append(y)
    n_liq.append(n)
    s_liq.append(s)
    t_data.append(t)
    p_data.append(p)
    jacobian.append(dparam)

Residual statistics

In [ ]:
print ('Average residual:', np.mean(np.array(y_data)))
print ('Std Dev residual:', np.std(np.array(y_data)))

In [ ]:
for i,y in enumerate(y_data):
    if y > 40000.0 or y < -20000.0:
        print ('{0:10.2f} {1:5s} {2:<10s} {3:10.2f} {4:10.2f} {5:7.3f}'.format(y, str(s_data[i]), name[i], t_data[i], p_data[i], y_ref[i]))

In [ ]:
results = sm.OLS(np.array(y_data), np.array(t_data)).fit()
results.summary()

Select a subset of data points to identify in plots (these are "natural" compositions - with iron and calcium)

In [ ]:
selection = [name.index('PPK(g)'), name.index('PK(g)'), name.index('KPK(g)'), name.index('K(g)')]

Estimated Zr concentration plotted against measured concentration

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(np.array([x for i,x in enumerate(y_ref) if y_con[i]]), 
         np.array([x for i,x in enumerate(y_est) if y_con[i]]), 'r+')
plt.plot(np.array([x for i,x in enumerate(y_ref) if not y_con[i]]), 
         np.array([x for i,x in enumerate(y_est) if not y_con[i]]), 'b+')
plt.plot(np.array([y_ref[i] for i in selection]), 
         np.array([y_est[i] for i in selection]), 'ko')
plt.plot(np.array([0,max(y_ref)]), np.array([0,max(y_ref)]), 'g-')
plt.ylabel('wt ZrO2 est')
plt.xlabel('wt ZrO2 data')
plt.subplot(1,2,2)
plt.plot(np.array([91.224*1000000.0*x/100.0/123.218 for i,x in enumerate(y_ref) if y_con[i]]), 
         np.array([91.224*1000000.0*x/100.0/123.218 for i,x in enumerate(y_est) if y_con[i]]), 'r+')
plt.plot(np.array([0,91.224*1000000.0*max(y_ref)/100.0/123.218]), np.array([0,91.224*1000000.0*max(y_ref)/100.0/123.218]), 'g-')
plt.ylabel('ppm ZrO2 est')
plt.xlabel('ppm ZrO2 data')
#axes = plt.gca()
#axes.set_xlim(0,1)
plt.tight_layout()
plt.show

Examine residuals (zircon affinity) for the Watson (1979) data set

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.plot([min(x_data),max(x_data)], [0.0,0.0], 'g-')
plt.plot(np.array(x_data), np.array(y_data), 'r+')
plt.plot(np.array([x_data[i] for i in selection]), np.array([y_data[i] for i in selection]), 'ko')
plt.ylabel('residual')
plt.xlabel('index')
plt.subplot(1,3,2)
plt.plot([min(np.array(n_liq)[:,0]),max(np.array(n_liq)[:,0])], [0.0,0.0], 'g-')
plt.plot(np.array(np.array(n_liq)[:,0]), np.array(y_data), 'r+')
plt.plot(np.array(np.array([n_liq[i] for i in selection])[:,0]), np.array([y_data[i] for i in selection]), 'ko')
plt.ylabel('residual')
plt.xlabel('X SiO2')
plt.subplot(1,3,3)
plt.plot([min(np.array(n_liq)[:,16]),max(np.array(n_liq)[:,16])], [0.0,0.0], 'g-')
plt.plot(np.array(np.array(n_liq)[:,16]), np.array(y_data), 'r+')
plt.plot(np.array(np.array([n_liq[i] for i in selection])[:,16]), np.array([y_data[i] for i in selection]), 'ko')
plt.ylabel('residual')
plt.xlabel('X ZrSiO4')
plt.tight_layout()
plt.show()

Examine residuals (zircon affinity) dependence on alkali content

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot([min(np.array(n_liq)[:,11]),max(np.array(n_liq)[:,11])], [0.0,0.0], 'g-')
plt.plot(np.array(np.array(n_liq)[:,11]), np.array(y_data), 'r+')
plt.plot(np.array(np.array([n_liq[i] for i in selection])[:,11]), np.array([y_data[i] for i in selection]), 'ko')
plt.ylabel('residual')
plt.xlabel('X Na2SiO3')
plt.subplot(1,2,2)
plt.plot([min(np.array(n_liq)[:,12]),max(np.array(n_liq)[:,12])], [0.0,0.0], 'g-')
plt.plot(np.array(np.array(n_liq)[:,12]), np.array(y_data), 'r+')
plt.plot(np.array(np.array([n_liq[i] for i in selection])[:,12]), np.array([y_data[i] for i in selection]), 'ko')
plt.ylabel('residual')
plt.xlabel('X KAlSiO4')
plt.tight_layout()
plt.show()

Examine residuals (zircon affinity) dependence on temperature

In [ ]:
plt.figure(figsize=(15,5))
plt.plot([min(t_data)-273.15,max(t_data)-273.15], [0.0,0.0], 'g-')
plt.plot(np.array(t_data)-273.15, np.array(y_data), 'r+')
plt.plot(np.array([t_data[i] for i in selection])-273.15, np.array([y_data[i] for i in selection]), 'ko')
plt.ylabel('residual')
plt.xlabel('T °C')
plt.show()

Examine Zr-speciation

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.plot(np.array(x_data), np.array(n_liq)[:,16], 'r+')
plt.plot(np.array([x_data[i] for i in selection]), np.array([n_liq[i] for i in selection])[:,16], 'ko')
plt.ylabel('n ZrSiO4')
plt.xlabel('index')
plt.subplot(1,3,2)
plt.plot(np.array(x_data), np.array(s_liq)[:,1], 'r+')
plt.plot(np.array([x_data[i] for i in selection]), np.array([s_liq[i] for i in selection])[:,1], 'ko')
plt.ylabel('n Na4ZrSi2O8')
plt.xlabel('index')
plt.subplot(1,3,3)
plt.plot(np.array(x_data), np.array(s_liq)[:,2], 'r+')
plt.plot(np.array([x_data[i] for i in selection]), np.array([s_liq[i] for i in selection])[:,2], 'ko')
plt.ylabel('n K4ZrSi2O8')
plt.xlabel('index')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.semilogy(np.array(x_data), np.array(n_liq)[:,16], 'r+', label='ZrSiO4')
plt.semilogy(np.array(x_data), np.array(s_liq)[:,1],  'g+', label='Na4ZrSi2O8')
plt.semilogy(np.array(x_data), np.array(s_liq)[:,2],  'b+', label='K4ZrSi2O8')
plt.semilogy(np.array([x_data[i] for i in selection]), np.array([n_liq[i] for i in selection])[:,16], 'ro')
plt.semilogy(np.array([x_data[i] for i in selection]), np.array([s_liq[i] for i in selection])[:,1],  'go')
plt.semilogy(np.array([x_data[i] for i in selection]), np.array([s_liq[i] for i in selection])[:,2],  'bo')
plt.ylabel('n Zr species')
plt.xlabel('index')
plt.legend()
plt.subplot(1,2,2)
plt.semilogy(np.array(x_data), np.array(s_liq)[:,1],  'g+')
plt.semilogy(np.array([x_data[i] for i in selection]), np.array([s_liq[i] for i in selection])[:,1],  'go')
plt.ylabel('n Na4ZrSi2O8')
plt.xlabel('index')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.semilogy(np.array(n_liq)[:,11], np.array(n_liq)[:,16], 'r+', label='ZrSiO4')
plt.semilogy(np.array(n_liq)[:,11], np.array(s_liq)[:,1],  'g+', label='Na4ZrSi2O8')
plt.semilogy(np.array(n_liq)[:,11], np.array(s_liq)[:,2],  'b+', label='K4ZrSi2O8')
plt.semilogy(np.array([n_liq[i] for i in selection])[:,11], np.array([n_liq[i] for i in selection])[:,16], 'ro')
plt.semilogy(np.array([n_liq[i] for i in selection])[:,11], np.array([s_liq[i] for i in selection])[:,1],  'go')
plt.semilogy(np.array([n_liq[i] for i in selection])[:,11], np.array([s_liq[i] for i in selection])[:,2],  'bo')
plt.ylabel('n Zr species')
plt.xlabel('n Na2SiO3')
plt.legend()
plt.subplot(1,2,2)
plt.semilogy(np.array(n_liq)[:,11], np.array(s_liq)[:,1],  'g+')
plt.semilogy(np.array([n_liq[i] for i in selection])[:,11], np.array([s_liq[i] for i in selection])[:,1],  'go')
plt.ylabel('n Na4ZrSi2O8')
plt.xlabel('n Na2SiO3')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.semilogy(np.array(n_liq)[:,12], np.array(n_liq)[:,16], 'r+', label='ZrSiO4')
plt.semilogy(np.array(n_liq)[:,12], np.array(s_liq)[:,1],  'g+', label='Na4ZrSi2O8')
plt.semilogy(np.array(n_liq)[:,12], np.array(s_liq)[:,2],  'b+', label='K4ZrSi2O8')
plt.semilogy(np.array([n_liq[i] for i in selection])[:,12], np.array([n_liq[i] for i in selection])[:,16], 'ro')
plt.semilogy(np.array([n_liq[i] for i in selection])[:,12], np.array([s_liq[i] for i in selection])[:,1],  'go')
plt.semilogy(np.array([n_liq[i] for i in selection])[:,12], np.array([s_liq[i] for i in selection])[:,2],  'bo')
plt.ylabel('n Zr species')
plt.xlabel('n KAlSiO4')
plt.legend()
plt.subplot(1,2,2)
plt.semilogy(np.array(n_liq)[:,12], np.array(s_liq)[:,2],  'b+')
plt.semilogy(np.array([n_liq[i] for i in selection])[:,12], np.array([s_liq[i] for i in selection])[:,2],  'bo')
plt.ylabel('n K4ZrSi2O8')
plt.xlabel('n KAlSiO4')
plt.tight_layout()
plt.show()

Select Watson (1979) data points for inclusion in final data fit

In [ ]:
#selection = [name.index('PPK(g)'), name.index('PK(g)'), name.index('KPK(g)'), name.index('K(g)')]
selection = []

In [ ]:
name_all   += [name[i] for i in selection]
x_data_all += [x_data[i] for i in selection]
y_data_all += [y_data[i] for i in selection]
n_liq_all  += [n_liq[i] for i in selection]
s_liq_all  += [s_liq[i] for i in selection]
t_data_all += [t_data[i] for i in selection]
p_data_all += [p_data[i] for i in selection]
s_data_all += [s_data[i] for i in selection]
y_est_all  += [y_est[i] for i in selection]
y_con_all  += [y_con[i] for i in selection]
y_ref_all  += [y_ref[i] for i in selection]

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(np.array([2*x[11]/x[12] for x in n_liq_all]), np.array([y[1]/y[2] for y in s_liq_all]), 'r+')
plt.plot(np.array([0,5]), np.array([0,5]), 'g-')
plt.ylabel('n Na4ZrSi2O8 / n K4ZrSi2O8')
plt.xlabel('n 2*Na2SiO3 / n KAlSiO4')
plt.show()

## Optimize parameters from experimental data sets
Note that this process resets the parameter array to the resulting optimization

In [ ]:
def res_func(x, t_data, p_data, n_liq, 
             ref_param_values, rMELTS_ZR, adj_H_ZrSiO4, adj_S_ZrSiO4, adj_V_ZrSiO4, use_weighting, 
             use_priors, prior_weight, 
             zirconium_min_affinities):
    y = []
    priors = []
    rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(5,  ref_param_values[ 5] + adj_H_ZrSiO4 - 225000.0 + x[0]) # Na4ZrSi2O8
    rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(6,  ref_param_values[ 6] + adj_S_ZrSiO4            + x[3])
    rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(7,  ref_param_values[ 7] + adj_V_ZrSiO4            + x[4])
    rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(8,  ref_param_values[ 8] + adj_H_ZrSiO4 -  75000.0 + x[1]) # K4ZrSi2O8
    rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(9,  ref_param_values[ 9] + adj_S_ZrSiO4            + x[3])
    rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(10, ref_param_values[10] + adj_V_ZrSiO4            + x[4])
    rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(11, ref_param_values[11] + adj_H_ZrSiO4            + x[2]) # ZrSiO4
    rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(12, ref_param_values[12] + adj_S_ZrSiO4            + x[3])
    rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(13, ref_param_values[13] + adj_V_ZrSiO4            + x[4])
    for t, p, n in zip(t_data, p_data, n_liq):
        mu = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_dgdn(t, p, n)
        s = rMELTS_ZR.cy_Liquid_rMELTS_ZR_order_params(t, p, n)
        affinity = zirconium_min_affinities(t, p, mu, print_results=False)[0]
        weight = 1.0 
        if use_weighting:
            weight = ((2*n[11]/n[12])/(s[1]/s[2]) + (s[1]/s[2])/(2*n[11]/n[12]))/2  # += (2*n[11]/n[12] - s[1]/s[2])**2
        y.append(affinity*weight)
        if use_priors:
            priors.append(prior_weight*np.log((2*n[11]/n[12])/(s[1]/s[2])))
    return np.array(y+priors)

Bayesian prior: $w_p \log \left( \frac{ \frac{2 n_{12}}{n_{13}} }{ \frac{y_{19}}{y_{20}} } \right)$ to impose Watson's Na/K observation

In [ ]:
use_weighting = False
use_priors = True
prior_weight = 2500.0
#
# perform fit
fit_res = sci.optimize.least_squares(res_func, 
                                     [0.0, 0.0, -50000.0, 0, 0],
                                     bounds=(np.array([ -np.inf, -np.inf, -np.inf, -np.inf, -np.inf]), 
                                             np.array([  np.inf,  np.inf,  np.inf,  np.inf,  np.inf])
                                            ),
                                     jac='3-point',
                                     method='trf',
                                     ftol=None, gtol=1.0e-8,
                                     args=(t_data_all, p_data_all, n_liq_all, ref_param_values, rMELTS_ZR, 
                                           adj_H_ZrSiO4, adj_S_ZrSiO4, adj_V_ZrSiO4, use_weighting, use_priors, 
                                           prior_weight, zirconium_min_affinities),
                                     verbose=2)
#
# recalculate residuals associated with the reaction affinity
y_data_rev = res_func(fit_res.x, t_data_all, p_data_all, n_liq_all, 
                      ref_param_values, rMELTS_ZR, adj_H_ZrSiO4, adj_S_ZrSiO4, adj_V_ZrSiO4, False, False, 
                      prior_weight, zirconium_min_affinities)
rms_residuals = np.sqrt(np.mean(y_data_rev**2))
#
# sigma is calculated from teh full Jacobian, including rows associated with priors
sigma = np.sqrt(np.diagonal(np.linalg.inv(2*np.matmul(fit_res.jac.T, fit_res.jac)))*rms_residuals**2)
print ('X:', fit_res.x)
print ('grad', fit_res.grad)
print ('Optimality:', fit_res.optimality)
print ('nfev:', fit_res.nfev)
print ('njev:', fit_res.njev)
print ('status:', fit_res.status)
print ('message:', fit_res.message)
print ('success:', fit_res.success)
print ('rms of affinity residuals', rms_residuals)
print ('H Na4ZrSi2O8 old: {0:12.2f} new: {1:12.2f} dif: {2:12.2f} param: {3:12.2f} sigma: {4:12.5g}'.format(
    ref_param_values[ 5] + adj_H_ZrSiO4 - 225000.0, 
    ref_param_values[ 5] + adj_H_ZrSiO4 - 225000.0 + fit_res.x[0],
    fit_res.x[0], - 225000.0 + fit_res.x[0], sigma[0]))
print ('H K4ZrSi2O8  old: {0:12.2f} new: {1:12.2f} dif: {2:12.2f} param: {3:12.2f} sigma: {4:12.5g}'.format(
    ref_param_values[ 8] + adj_H_ZrSiO4 -  75000.0, 
    ref_param_values[ 8] + adj_H_ZrSiO4 -  75000.0 + fit_res.x[1], 
    fit_res.x[1], -  75000.0 + fit_res.x[1], sigma[1]))
print ('H ZrSiO4     old: {0:12.2f} new: {1:12.2f} dif: {2:12.2f} param: {3:12.2f} sigma: {4:12.5g}'.format(
    ref_param_values[11] + adj_H_ZrSiO4,
    ref_param_values[11] + adj_H_ZrSiO4 + fit_res.x[2],
    fit_res.x[2], adj_H_ZrSiO4 + fit_res.x[2], sigma[2]))
print ('S ZrSiO4     old: {0:12.2f} new: {1:12.2f} dif: {2:12.2f} param: {3:12.2f} sigma: {4:12.5g}'.format(
    ref_param_values[12] + adj_S_ZrSiO4,
    ref_param_values[12] + adj_S_ZrSiO4 + fit_res.x[3],
    fit_res.x[3], adj_S_ZrSiO4 + fit_res.x[3], sigma[3]))
print ('V ZrSiO4     old: {0:12.2f} new: {1:12.2f} dif: {2:12.2f} param: {3:12.2f} sigma: {4:12.5g}'.format(
    ref_param_values[13] + adj_V_ZrSiO4,
    ref_param_values[13] + adj_V_ZrSiO4 + fit_res.x[4],
    fit_res.x[4], adj_V_ZrSiO4 + fit_res.x[4], sigma[4]))
print ('S Na4ZrSi2O8 old: {0:12.2f} new: {1:12.2f} dif: {2:12.2f} param: {3:12.2f} sigma: {4:12.5g}'.format(
    ref_param_values[ 6] + adj_S_ZrSiO4,
    ref_param_values[ 6] + adj_S_ZrSiO4 + fit_res.x[3],
    fit_res.x[3], 0.0, sigma[3]))
print ('V Na4ZrSi2O8 old: {0:12.2f} new: {1:12.2f} dif: {2:12.2f} param: {3:12.2f} sigma: {4:12.5g}'.format(
    ref_param_values[ 7] + adj_V_ZrSiO4,
    ref_param_values[ 7] + adj_V_ZrSiO4 + fit_res.x[4],
    fit_res.x[4], 0.0, sigma[4]))
print ('S K4ZrSi2O8  old: {0:12.2f} new: {1:12.2f} dif: {2:12.2f} param: {3:12.2f} sigma: {4:12.5g}'.format(
    ref_param_values[ 9] + adj_S_ZrSiO4,
    ref_param_values[ 9] + adj_S_ZrSiO4 + fit_res.x[3],
    fit_res.x[3], 0.0, sigma[3]))
print ('V K4ZrSi2O8  old: {0:12.2f} new: {1:12.2f} dif: {2:12.2f} param: {3:12.2f} sigma: {4:12.5g}'.format(
    ref_param_values[10] + adj_V_ZrSiO4,
    ref_param_values[10] + adj_V_ZrSiO4 + fit_res.x[4],
    fit_res.x[4], 0.0, sigma[4]))

Construct a parameter correlation matrix from the Jacobian evaluated at the optimal solution

In [ ]:
print ('Estimated parameter correlation matrix:')
cov_mat = np.linalg.inv(2*np.matmul(fit_res.jac.T, fit_res.jac))*rms_residuals**2
labl = ['', 'H Na4', 'H K4', 'H Zr', 'S Zr', 'V Zr']
for x in labl:
    print ('{0:>12.12s}'.format(x), end=' ')
print ('')
for i,row in enumerate(cov_mat):
    print ('{0:>12.12s}'.format(labl[i+1]), end=' ')
    for j,col in enumerate(row):
        print ('{0:12.6g}'.format(col/sigma[i]/sigma[j]), end=' ')
    print ('')

Decompose the Jacobian (evaluated at the optimal solution) using singular value analysis

In [ ]:
u,s,vT = np.linalg.svd(fit_res.jac)
for col in s:
    print ('{0:13.6g}'.format(col), end=' ')
print ('')
print ('')
for row in vT.T:
    for col in row:
        print ('{0:13.6g}'.format(col), end=' ')
    print ('')

Residual statistics (compared to guessed parameter fit)

In [ ]:
print ('Average residual before: {0:13.6g} fit: {1:13.6g} fit+prior: {2:13.6g}'.format(np.mean(np.array(y_data_all)), np.mean(y_data_rev), np.mean(fit_res.fun)))
print ('Std Dev residual before: {0:13.6g} fit: {1:13.6g} fit+prior: {2:13.6g}'.format(np.std(np.array(y_data_all)),  np.std(y_data_rev),  np.std(fit_res.fun)))

Monte Carlo analysis of parameter variance

In [ ]:
deviates = np.random.multivariate_normal(fit_res.x, cov_mat, size=100)
plt.figure(figsize=(20,5))
plt.subplot(1,5,1)
plt.hist(deviates[:,0], orientation='horizontal')
plt.title('H Na4ZrSi2O8')
plt.subplot(1,5,2)
plt.hist(deviates[:,1], orientation='horizontal')
plt.title('H K4ZrSi2O8')
plt.subplot(1,5,3)
plt.hist(deviates[:,2], orientation='horizontal')
plt.title('H ZrSiO4')
plt.subplot(1,5,4)
plt.hist(deviates[:,3], orientation='horizontal')
plt.title('S ZrSiO4')
plt.subplot(1,5,5)
plt.hist(deviates[:,4], orientation='horizontal')
plt.title('V ZrSiO4')
plt.tight_layout()
plt.show()

Recompute speciation and plot Na/K ratio of Zr species against Na/K bulk composition ratio

In [ ]:
s_revised = []
for t, p, n in zip(t_data_all, p_data_all, n_liq_all):
    s_revised.append(rMELTS_ZR.cy_Liquid_rMELTS_ZR_order_params(t, p, n))
s_revised = s_revised
fig = plt.figure(figsize=(15,10))
plt.loglog(np.array([2*x[11]/x[12] for x in n_liq_all]), np.array([y[1]/y[2] for y in s_revised]), 'r+')
plt.loglog(np.array([0,5]), np.array([0,5]), 'g-')
plt.ylabel('Na/K species ratio')
plt.xlabel('Na/K reported ratio')
plt.grid(True)
plt.xticks([0.5,0.6,0.7,0.8,0.9,1,2,3,4,5],
           ['','0.6','0.7','0.8','0.9','1','2','3','4','5'])
plt.yticks([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100],
          ['0.1','','','','0.5','','','','','1','','','','5','','','','','10','','','','50','','','','','100'])
#plt.ylim(0,5)
plt.show()
fig.savefig("Figure_Na_K.pdf", bbox_inches='tight')

Estimate zirconium content by zeroing the residuals

In [ ]:
Zr_meas = []
Zr_pred = []
def residual(log_Zr_est, t, p, nLiq):
    Zr_est = np.exp(log_Zr_est)
    wtZrO2_est = 100.0*123.218*Zr_est/91.224/1000000.0
    elm_est = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_conv_moles_to_elm(nLiq)
    Zr_old = elm_est[core.chem.PERIODIC_ORDER.tolist().index('Zr')]
    elm_est[core.chem.PERIODIC_ORDER.tolist().index('Zr')] -= Zr_old
    elm_est[core.chem.PERIODIC_ORDER.tolist().index('O')]  -= 2.0*Zr_old
    elm_est[core.chem.PERIODIC_ORDER.tolist().index('Zr')] += wtZrO2_est/123.218
    elm_est[core.chem.PERIODIC_ORDER.tolist().index('O')]  += wtZrO2_est*2.0/123.218
    n_est = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_conv_elm_to_moles(elm_est)
    mu_est= rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_dgdn(t,p,n_est)
    result = zirconium_min_affinities(t, p, mu_est, print_results=False)[0]
    return result
#
for t_in, p_in, n_in, y_ref in zip(t_data_all, p_data_all, n_liq_all, y_ref_all):
    try:
        sol = sci.optimize.root_scalar(residual, 
                                       args=(t_in, p_in, n_in),
                                       rtol=1.e-5,
                                       bracket=[0,10],
                                       method='brentq',
                                       maxiter=50)
        print (sol)
        Zr_pred.append(np.exp(sol.root))
        Zr_meas.append(91.224*1000000.0*y_ref/123.218/100.0)
    except ValueError:
        print ('Solution not within brackets')

In [ ]:
fig = plt.figure(figsize=(15,10))
#plt.subplot(1,2,1)
plt.loglog(np.array(Zr_meas), np.array(Zr_pred), 'r+')
plt.loglog(np.array([100,10000]), np.array([100, 10000]), 'g-')
plt.ylabel('Zr PPM predicted value')
plt.xlabel('Zr PPM reported value')
plt.grid(True)
plt.xticks([100,200,300,400,500,600,700,800,900,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000],
           ['100','','','','500','','','','','1000','','','','5000','','','','','10000'])
plt.yticks([100,200,300,400,500,600,700,800,900,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000],
           ['100','','','','500','','','','','1000','','','','5000','','','','','10000'])
#plt.tight_layout()
plt.show()
fig.savefig("Figure_Zr_Zr.pdf", bbox_inches='tight')

If the priors constraints are imposed, compare affinity resiudlas and prior residuals

In [ ]:
if use_priors:
    jac_rows = int(fit_res.fun.shape[0]/2)
    fig = plt.figure(figsize=(15,10))
    plt.subplot(2,2,1)
    plt.plot(np.array(n_liq_all)[:,11], fit_res.fun[:jac_rows], 'r+', label='affin')
    plt.plot(np.array(n_liq_all)[:,11], fit_res.fun[jac_rows:], 'b+', label='prior')
    plt.xlabel('n Na2SiO3')
    plt.ylabel('residual')
    plt.grid(True)
    plt.legend()
    plt.subplot(2,2,2)
    plt.plot(np.array(n_liq_all)[:,12], fit_res.fun[:jac_rows], 'r+', label='affin')
    plt.plot(np.array(n_liq_all)[:,12], fit_res.fun[jac_rows:], 'b+', label='prior')
    plt.xlabel('n KAlSiO4')
    plt.ylabel('residual')
    plt.grid(True)
    plt.legend()
    plt.subplot(2,2,3)
    plt.plot(np.array(t_data_all), fit_res.fun[:jac_rows], 'r+', label='affin')
    plt.plot(np.array(t_data_all), fit_res.fun[jac_rows:], 'b+', label='prior')
    plt.xlabel('T (K)')
    plt.ylabel('residual')
    plt.grid(True)
    plt.legend()
    plt.subplot(2,2,4)
    plt.plot(np.array(p_data_all), fit_res.fun[:jac_rows], 'r+', label='affin')
    plt.plot(np.array(p_data_all), fit_res.fun[jac_rows:], 'b+', label='prior')
    plt.xlabel('P (bars)')
    plt.ylabel('residual')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show
    fig.savefig("Figure_Residuals.pdf", bbox_inches='tight')

Compared guessed parameter residuals (x axis) against optimized parameter residuals (y axis)

In [ ]:
plt.figure(figsize=(15,5))
#plt.subplot(1,2,1)
plt.plot(np.array([x for i,x in enumerate(y_data_all) if auth_dat[i] == 1]), 
         np.array([x for i,x in enumerate(y_data_rev) if auth_dat[i] == 1]), 'r+')
plt.plot(np.array([x for i,x in enumerate(y_data_all) if auth_dat[i] == 2]), 
         np.array([x for i,x in enumerate(y_data_rev) if auth_dat[i] == 2]), 'b+')
plt.plot(np.array([x for i,x in enumerate(y_data_all) if auth_dat[i] == 3]), 
         np.array([x for i,x in enumerate(y_data_rev) if auth_dat[i] == 3]), 'k+')
plt.plot(np.array([y_data_all[i] for i in selection]), 
         np.array([fit_res.fun[i] for i in selection]), 'ko')
plt.plot(np.array([-10000,10000]), np.array([-10000, 10000]), 'g-')
plt.ylabel('fit residual')
plt.xlabel('inp residual')
#plt.tight_layout()
plt.show()

Compare temperature correlation to residuals (zircon affinities) calculated using parameter guesses and parameter fit

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(np.array([x for i,x in enumerate(t_data_all) if auth_dat[i] == 1]), 
         np.array([x for i,x in enumerate(y_data_all) if auth_dat[i] == 1]), 'r+')
plt.plot(np.array([x for i,x in enumerate(t_data_all) if auth_dat[i] == 2]), 
         np.array([x for i,x in enumerate(y_data_all) if auth_dat[i] == 2]), 'b+')
plt.plot(np.array([x for i,x in enumerate(t_data_all) if auth_dat[i] == 3]), 
         np.array([x for i,x in enumerate(y_data_all) if auth_dat[i] == 3]), 'k+')
plt.plot(np.array([t_data_all[i] for i in selection]), 
         np.array([y_data_all[i] for i in selection]), 'ko')
plt.plot(np.array([min(t_data_all), max(t_data_all)]), np.array([0, 0]), 'g-')
plt.ylabel('fit residual')
plt.xlabel('T (K)')
plt.subplot(1,2,2)
plt.plot(np.array([x for i,x in enumerate(t_data_all) if auth_dat[i] == 1]), 
         np.array([x for i,x in enumerate(y_data_rev) if auth_dat[i] == 1]), 'r+')
plt.plot(np.array([x for i,x in enumerate(t_data_all) if auth_dat[i] == 2]), 
         np.array([x for i,x in enumerate(y_data_rev) if auth_dat[i] == 2]), 'b+')
plt.plot(np.array([x for i,x in enumerate(t_data_all) if auth_dat[i] == 3]), 
         np.array([x for i,x in enumerate(y_data_rev) if auth_dat[i] == 3]), 'k+')
plt.plot(np.array([t_data_all[i] for i in selection]), 
         np.array([fit_res.fun[i] for i in selection]), 'ko')
plt.plot(np.array([min(t_data_all), max(t_data_all)]), np.array([0, 0]), 'g-')
plt.ylabel('fit residual')
plt.xlabel('T (K)')
plt.tight_layout()
plt.show()

Compare pressure correlation to residuals (zircon affinities) calculated using parameter guesses and parameter fit

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(np.array([x for i,x in enumerate(p_data_all) if auth_dat[i] == 1]), 
         np.array([x for i,x in enumerate(y_data_all) if auth_dat[i] == 1]), 'r+')
plt.plot(np.array([x for i,x in enumerate(p_data_all) if auth_dat[i] == 2]), 
         np.array([x for i,x in enumerate(y_data_all) if auth_dat[i] == 2]), 'b+')
plt.plot(np.array([x for i,x in enumerate(p_data_all) if auth_dat[i] == 3]), 
         np.array([x for i,x in enumerate(y_data_all) if auth_dat[i] == 3]), 'k+')
plt.plot(np.array([min(p_data_all), max(p_data_all)]), np.array([0, 0]), 'g-')
plt.ylabel('fit residual')
plt.xlabel('P (bars)')
plt.subplot(1,2,2)
plt.plot(np.array([x for i,x in enumerate(p_data_all) if auth_dat[i] == 1]), 
         np.array([x for i,x in enumerate(y_data_rev) if auth_dat[i] == 1]), 'r+')
plt.plot(np.array([x for i,x in enumerate(p_data_all) if auth_dat[i] == 2]), 
         np.array([x for i,x in enumerate(y_data_rev) if auth_dat[i] == 2]), 'b+')
plt.plot(np.array([x for i,x in enumerate(p_data_all) if auth_dat[i] == 3]), 
         np.array([x for i,x in enumerate(y_data_rev) if auth_dat[i] == 3]), 'k+')
plt.plot(np.array([min(p_data_all), max(p_data_all)]), np.array([0, 0]), 'g-')
plt.ylabel('fit residual')
plt.xlabel('P (bars)')
plt.tight_layout()
plt.show()

## Plot zircon saturation surface

**Assemble a dictionary of interesting compositions**  

- Bishop Tuff rhyolite composition is LBT from Gualda
- Pantellerite composition is from Carmichael ISA, Turner FJ, Verhoogen J (1974) Igneous Petrology, McGraw-Hill, New York, 739pp; page 237, Table 5-1, column 1. Water content estimated at 2.5 wt% from ion-probe data reported in Lowenstern JB, Mahood GA (1991) New data on magmatic H<sub>2</sub>O contents of pantellerites, with implications for petrogenesis and eruptive dynamics of Pantelleria. Bulletin of Volcanology, 54, 78-83
- Carmichael reports Zr concentration in the pantellerite is 1800 PPM 
- Trachyte, Phonolite in Carmichael, Turner and Verhoogen (1974), p36, Table 2-3
- Dacite, in Carmichael, Turner and Verhoogen (1974), page 9
- Madupite is LH.16 in Carmichael ISE (1967) The Mineralogy and Petrology of the Volcanic Rocks from the Leucite Hills, Wyoming. Contributions to Mineralogy and Petrology, 15, 24-66. It contains appreciable wadeite, Zr<sub>2</sub>K<sub>4</sub>Si<sub>6</sub>O<sub>18</sub>.  This rock has 0.27 wt% ZrO<sub>2</sub>. The Wyomingite is LH.3 . The rock has no Zr analysis but other wyomingites have 0.28 wt% ZrO<sub>2</sub>.  All the analyses are from Table 12 (page 50).

In [ ]:
composition_d = {
    'rhyolite': { # Bishop Tuff
        'SiO2':  77.5, 
        'TiO2':   0.08, 
        'Al2O3': 12.5, 
        'Fe2O3':  0.207,
        'Cr2O3':  0.0, 
        'FeO':    0.473, 
        'MnO':    0.0,
        'MgO':    0.03, 
        'NiO':    0.0, 
        'CoO':    0.0,
        'CaO':    0.43, 
        'Na2O':   3.98, 
        'K2O':    4.88, 
        'P2O5':   0.0, 
        'H2O':    5.5,
        'CO2':    0.05
    },
    'Iceland-rhyolite': { # Carmichael, Turner and Verhoogen, p 237; 385 PPM Zr (p. 75)
        'SiO2':  74.96, 
        'TiO2':   0.23, 
        'Al2O3': 12.55, 
        'Fe2O3':  1.72,
        'Cr2O3':  0.0, 
        'FeO':    0.71, 
        'MnO':    0.04,
        'MgO':    0.02, 
        'NiO':    0.0, 
        'CoO':    0.0,
        'CaO':    0.90, 
        'Na2O':   4.41, 
        'K2O':    3.65, 
        'P2O5':   0.04, 
        'H2O':    5.5,
        'CO2':    0.05
    },
    'California-rhyolite': { # Carmichael, Turner and Verhoogen, p 237
        'SiO2':  75.04, 
        'TiO2':   0.07, 
        'Al2O3': 12.29, 
        'Fe2O3':  0.33,
        'Cr2O3':  0.0, 
        'FeO':    0.71, 
        'MnO':    0.05,
        'MgO':    0.04, 
        'NiO':    0.0, 
        'CoO':    0.0,
        'CaO':    0.58, 
        'Na2O':   4.03, 
        'K2O':    4.66, 
        'P2O5':   0.01, 
        'H2O':    5.5,
        'CO2':    0.05
    },
    'dacite': {
        'SiO2':  63.58, 
        'TiO2':   0.64, 
        'Al2O3': 16.67, 
        'Fe2O3':  2.24,
        'Cr2O3':  0.0, 
        'FeO':    3.00, 
        'MnO':    0.11,
        'MgO':    2.12, 
        'NiO':    0.0, 
        'CoO':    0.0,
        'CaO':    5.53, 
        'Na2O':   3.98, 
        'K2O':    1.40, 
        'P2O5':   0.0, 
        'H2O':    3.00,
        'CO2':    0.05
    },
    'pantellerite': { # Pantellerite, Carmichael Turner and Verhoogen, p 237
        'SiO2':  69.81, 
        'TiO2':   0.45, 
        'Al2O3':  8.59, 
        'Fe2O3':  2.28,
        'Cr2O3':  0.0, 
        'FeO':    5.76, 
        'MnO':    0.28,
        'MgO':    0.10, 
        'NiO':    0.0, 
        'CoO':    0.0,
        'CaO':    0.42, 
        'Na2O':   6.46, 
        'K2O':    4.49, 
        'P2O5':   0.0, 
        'H2O':    2.5,
        'CO2':    0.05
    },
    'comendite': { # Comendite, Carmichael Turner and Verhoogen, p 237
        'SiO2':  73.06, 
        'TiO2':   0.23, 
        'Al2O3':  9.76, 
        'Fe2O3':  2.74,
        'Cr2O3':  0.0, 
        'FeO':    2.70, 
        'MnO':    0.13,
        'MgO':    0.10, 
        'NiO':    0.0, 
        'CoO':    0.0,
        'CaO':    0.32, 
        'Na2O':   5.64, 
        'K2O':    4.34, 
        'P2O5':   0.02, 
        'H2O':    2.5,
        'CO2':    0.05
    },
    'trachyte': {
        'SiO2':  57.73, 
        'TiO2':   1.18, 
        'Al2O3': 17.05, 
        'Fe2O3':  2.55,
        'Cr2O3':  0.0, 
        'FeO':    4.35, 
        'MnO':    0.28,
        'MgO':    1.11, 
        'NiO':    0.0, 
        'CoO':    0.0,
        'CaO':    3.10, 
        'Na2O':   6.81, 
        'K2O':    4.27, 
        'P2O5':   0.34, 
        'H2O':    1.0,
        'CO2':    0.05
    },
    'phonolite': {
        'SiO2':  54.55, 
        'TiO2':   1.60, 
        'Al2O3': 19.00, 
        'Fe2O3':  1.75,
        'Cr2O3':  0.0, 
        'FeO':    3.78, 
        'MnO':    0.16,
        'MgO':    1.76, 
        'NiO':    0.0, 
        'CoO':    0.0,
        'CaO':    4.07, 
        'Na2O':   9.06, 
        'K2O':    3.64, 
        'P2O5':   0.20, 
        'H2O':    1.0,
        'CO2':    0.05
    },
    'madupite': { # Leucite Hills, Wyoming
        'SiO2':  43.56, 
        'TiO2':   2.31, 
        'Al2O3':  7.85, 
        'Fe2O3':  5.57,
        'Cr2O3':  0.04, 
        'FeO':    0.85, 
        'MnO':    0.15,
        'MgO':   11.03, 
        'NiO':    0.0, 
        'CoO':    0.0,
        'CaO':   11.89, 
        'Na2O':   0.74, 
        'K2O':    7.19, 
        'P2O5':   1.50, 
        'H2O':    1.0,
        'CO2':    0.05
    },
    'wyomingite': { # Leucite Hills, Wyoming
        'SiO2':  50.23, 
        'TiO2':   2.27, 
        'Al2O3': 11.22, 
        'Fe2O3':  3.34,
        'Cr2O3':  0.0, 
        'FeO':    1.84, 
        'MnO':    0.05,
        'MgO':    7.09, 
        'NiO':    0.0, 
        'CoO':    0.0,
        'CaO':    5.99, 
        'Na2O':   1.37, 
        'K2O':    9.81, 
        'P2O5':   1.89, 
        'H2O':    2.0,
        'CO2':    0.05
    }
}

Define a function to generate a zircon or badeyelite saturation plot

In [ ]:
def generate_saturation_plot(phase='zircon', 
                             use_composition='rhyolite',
                             p=2000.0,
                             tmin=700.0,
                             tmax=1000.0,
                             tinc=25,
                             ref_Zr_l=None, # list
                             Zr_PPM_root_min=10.0,
                             Zr_PPM_root_max=500000.0,
                             print_steps=True,
                             debug=False,
                             deviates=None,
                             deviate_plot_quantiles=False
                            ):
    global composition_d, core, plt, sci, np, rMELTS_ZR
    #
    grm_oxides = composition_d[use_composition]
    elm_sys = ['H','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni']
    tot_grm_oxides = 0.0
    for key in grm_oxides.keys():
        tot_grm_oxides += grm_oxides[key]
    mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
    mol_elm = core.chem.mol_oxide_to_elem(mol_oxides)
    elm_ref = np.zeros(107)
    for i,sym in enumerate(
        ['Si', 'Ti', 'Al', 'Fe', 'Cr', 'Mn', 'Mg', 'Ni', 'Co', 'Ca', 'Na', 'K', 'P', 'H', 'C', 'O']):
        index = core.chem.PERIODIC_ORDER.tolist().index(sym)
        elm_ref[index] = mol_elm[i]
    #
    # Function used by minimizer
    def zero(x):
        global rMELTS_ZR
        elm = np.copy(elm_ref)
        Zr_ppm   = x
        wtZrO2 = 123.218*Zr_ppm/91.224/1000000.0
        wtZrO2 *= tot_grm_oxides + wtZrO2
        elm[core.chem.PERIODIC_ORDER.tolist().index('Zr')] += wtZrO2/123.218
        elm[core.chem.PERIODIC_ORDER.tolist().index('O')]  += wtZrO2*2.0/123.218
        n = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_conv_elm_to_moles(elm)
        if not rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_test_moles(n):
            print ('Output of intrinsic composition calculation fails tests for permissible values.')
        mu = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_dgdn(t,p,n)
        y = zirconium_min_affinities(t, p, mu, print_results=False)[0 if phase == 'zircon' else 1]
        if debug:
            print (t, Zr_ppm, y)
            print (n)
        return y
    #
    T_array = np.linspace(tmin, tmax, tinc, endpoint=True)
    if deviates is not None:
        remember_param_values = rMELTS_ZR.cy_Liquid_rMELTS_ZR_get_param_values()
        yPlot_all = []
        species_all = []
        for irow,row in enumerate(deviates): 
            if np.mod(irow,10) == 0:
                print ('Processing deviate:', irow)
            rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(5,  ref_param_values[ 5] + adj_H_ZrSiO4 - 225000.0 + row[0]) # Na4ZrSi2O8
            rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(6,  ref_param_values[ 6] + adj_S_ZrSiO4            + row[3])
            rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(7,  ref_param_values[ 7] + adj_V_ZrSiO4            + row[4])
            rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(8,  ref_param_values[ 8] + adj_H_ZrSiO4 -  75000.0 + row[1]) # K4ZrSi2O8
            rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(9,  ref_param_values[ 9] + adj_S_ZrSiO4            + row[3])
            rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(10, ref_param_values[10] + adj_V_ZrSiO4            + row[4])
            rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(11, ref_param_values[11] + adj_H_ZrSiO4            + row[2]) # ZrSiO4
            rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(12, ref_param_values[12] + adj_S_ZrSiO4            + row[3])
            rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(13, ref_param_values[13] + adj_V_ZrSiO4            + row[4])
            yPlot = []
            species = []
            for tc in T_array:
                t = tc + 273.15
                y = zero(100)
                sol = sci.optimize.root_scalar(zero, bracket=[Zr_PPM_root_min, Zr_PPM_root_max])
                yPlot.append(sol.root)
                elm = np.copy(elm_ref)
                wtZrO2 = 123.218*sol.root/91.224/1000000.0
                wtZrO2 *= tot_grm_oxides + wtZrO2
                elm[core.chem.PERIODIC_ORDER.tolist().index('Zr')] += wtZrO2/123.218
                elm[core.chem.PERIODIC_ORDER.tolist().index('O')]  += wtZrO2*2.0/123.218
                n = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_conv_elm_to_moles(elm)
                s = rMELTS_ZR.cy_Liquid_rMELTS_ZR_order_params(t, p, n)
                tot = n[16] + s[1] + s[2]
                species.append(np.array([100*n[16]/tot, 100*s[1]/tot, 100*s[2]/tot]))
            #
            yPlot_all.append(np.array(yPlot))
            species_all.append(np.array(species))
        yPlot = np.transpose(np.array(yPlot_all))
        species = np.array(species_all)
        rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_values(remember_param_values)
    else:
        yPlot = []
        species = []
        for tc in T_array:
            if print_steps:
                print ('{0:7.1f}'.format(tc), end=' ')
            t = tc + 273.15
            y = zero(100)
            sol = sci.optimize.root_scalar(zero, bracket=[Zr_PPM_root_min, Zr_PPM_root_max])
            yPlot.append(sol.root)
            #
            elm = np.copy(elm_ref)
            wtZrO2 = 123.218*sol.root/91.224/1000000.0
            wtZrO2 *= tot_grm_oxides + wtZrO2
            elm[core.chem.PERIODIC_ORDER.tolist().index('Zr')] += wtZrO2/123.218
            elm[core.chem.PERIODIC_ORDER.tolist().index('O')]  += wtZrO2*2.0/123.218
            n = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_conv_elm_to_moles(elm)
            s = rMELTS_ZR.cy_Liquid_rMELTS_ZR_order_params(t, p, n)
            tot = n[16] + s[1] + s[2]
            species.append(np.array([100*n[16]/tot, 100*s[1]/tot, 100*s[2]/tot]))
            if print_steps:
                print ('{0:10.1f} {1:1.1s} {2:2d} {3:6.2f} {4:6.2f} {5:6.2f}'.format(
                    sol.root, 'T' if sol.converged else 'F', sol.iterations, 100*n[16]/tot, 100*s[1]/tot, 100*s[2]/tot
                ))
        #
        yPlot = np.array(yPlot)
        species = np.array(species)
    #
    fig = plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    plt.title(phase+' saturation for '+use_composition+' composition')
    if deviate_plot_quantiles and deviates is not None:
        plt.plot(T_array, np.quantile(yPlot,0.50,axis=1), 'r-',  label='mean')
        plt.plot(T_array, np.quantile(yPlot,0.05,axis=1), 'r--', label='5 %')
        plt.plot(T_array, np.quantile(yPlot,0.95,axis=1), 'r--', label='95 %')
        plt.legend()
    else:
        plt.plot(T_array, yPlot, 'r-')
    if ref_Zr_l is not None:
        for x in ref_Zr_l:
            plt.plot(np.array([tmin, tmax]), np.array([x,x]), 'g-')
    plt.ylabel('PPM Zr')
    plt.xlabel('T °C')
    #
    plt.subplot(1,2,2)
    plt.title('Zr speciation for '+use_composition+' composition')
    if deviates is not None:
        if deviate_plot_quantiles:
            plt.plot(T_array, np.quantile(species[:,:,0],0.50,axis=0), 'r-',  label='ZrSiO4 mean')
            plt.plot(T_array, np.quantile(species[:,:,0],0.05,axis=0), 'r--', label='ZrSiO4 5%')
            plt.plot(T_array, np.quantile(species[:,:,0],0.95,axis=0), 'r--', label='ZrSiO4 95%')
            plt.plot(T_array, np.quantile(species[:,:,1],0.50,axis=0), 'g-',  label='Na4ZrSi2O8 mean')
            plt.plot(T_array, np.quantile(species[:,:,1],0.05,axis=0), 'g--', label='Na4ZrSi2O8 5%')
            plt.plot(T_array, np.quantile(species[:,:,1],0.95,axis=0), 'g--', label='Na4ZrSi2O8 95%')
            plt.plot(T_array, np.quantile(species[:,:,2],0.50,axis=0), 'b-',  label='K4ZrSi2O8 mean')
            plt.plot(T_array, np.quantile(species[:,:,2],0.05,axis=0), 'b--', label='K4ZrSi2O8 5%')
            plt.plot(T_array, np.quantile(species[:,:,2],0.95,axis=0), 'b--', label='K4ZrSi2O8 95%')
        else:
            for i in range(0,deviates.shape[0]):
                if i == 0:
                    plt.plot(T_array, species[i,:,0], 'r-', label='ZrSiO4')
                    plt.plot(T_array, species[i,:,1], 'g-', label='Na4ZrSi2O8')
                    plt.plot(T_array, species[i,:,2], 'b-', label='K4ZrSi2O8')
                else:
                    plt.plot(T_array, species[i,:,0], 'r-')
                    plt.plot(T_array, species[i,:,1], 'g-')
                    plt.plot(T_array, species[i,:,2], 'b-')
    else:
        plt.plot(T_array, species[:,0], 'r-', label='ZrSiO4')
        plt.plot(T_array, species[:,1], 'g-', label='Na4ZrSi2O8')
        plt.plot(T_array, species[:,2], 'b-', label='K4ZrSi2O8')
    plt.ylabel('% Zr-species')
    plt.xlabel('T °C')
    plt.legend()
    plt.tight_layout()
    plt.show()
    fig.savefig(phase + "_sat_" + use_composition + ".pdf", bbox_inches='tight')
    return (T_array, yPlot, species)

Make some plots of zircon saturation for various compositions

In [ ]:
T_array, yPlot, species = generate_saturation_plot(use_composition='rhyolite', ref_Zr_l=[80, 100, 120], tmin=700, tmax=800, print_steps=False)

In [ ]:
T_array, yPlot, species = generate_saturation_plot(use_composition='rhyolite', ref_Zr_l=[80, 100, 120], tmin=700, tmax=800, print_steps=False, 
                                                   deviates=deviates, deviate_plot_quantiles=True)

In [ ]:
T_array, yPlot, species = generate_saturation_plot(use_composition='California-rhyolite', ref_Zr_l=[110], tmax=800, print_steps=False, 
                                                   deviates=deviates, deviate_plot_quantiles=True)

In [ ]:
T_array, yPlot, species = generate_saturation_plot(use_composition='Iceland-rhyolite', ref_Zr_l=[385], print_steps=False, 
                                                   deviates=deviates, deviate_plot_quantiles=True)

In [ ]:
T_array, yPlot, species = generate_saturation_plot(use_composition='dacite', tmin=700, ref_Zr_l=[150], Zr_PPM_root_min=1.0,
                                                   print_steps=False, debug=False, deviates=deviates, deviate_plot_quantiles=True)

In [ ]:
T_array, yPlot, species = generate_saturation_plot(use_composition='pantellerite', ref_Zr_l=[1800], 
                                                   tmin=700, tmax=900, print_steps=False, 
                                                   deviates=deviates, deviate_plot_quantiles=True)

In [ ]:
T_array, yPlot, species = generate_saturation_plot(use_composition='comendite', ref_Zr_l=[1250], print_steps=False,
                                                  deviates=deviates, deviate_plot_quantiles=True)

In [ ]:
T_array, yPlot, species = generate_saturation_plot(use_composition='trachyte', ref_Zr_l=None, print_steps=False, 
                                                   deviates=deviates, deviate_plot_quantiles=True)

In [ ]:
T_array, yPlot, species = generate_saturation_plot(use_composition='phonolite', ref_Zr_l=None, print_steps=False, 
                                                   deviates=deviates, deviate_plot_quantiles=True)

This rock is saturated with a wadeite solid solution

In [ ]:
T_array, yPlot, species = generate_saturation_plot(use_composition='madupite', tmin=700.0, tmax=1200.0, ref_Zr_l=[2073],
                                                   Zr_PPM_root_min=1.0, Zr_PPM_root_max=250000.0, print_steps=False, 
                                                   deviates=deviates, deviate_plot_quantiles=True)

This rock is saturated with wadeite, Zr<sub>2</sub>K<sub>4</sub>Si<sub>6</sub>O<sub>18</sub> in solid solution with its Ti-equivalent, Ti<sub>2</sub>K<sub>4</sub>Si<sub>6</sub>O<sub>18</sub>

In [ ]:
T_array, yPlot, species = generate_saturation_plot(use_composition='wyomingite', tmin=700.0, tmax=1200.0, ref_Zr_l=[2073], 
                                                   Zr_PPM_root_min=10.0, Zr_PPM_root_max=500000.0, print_steps=False, debug=False,
                                                   deviates=deviates, deviate_plot_quantiles=True)

In [ ]:
T_array, yPlot, species = generate_saturation_plot(phase='badelyite', use_composition='wyomingite', tmin=700.0, tmax=1200.0, ref_Zr_l=[2073], 
                                                   Zr_PPM_root_min=10.0, Zr_PPM_root_max=500000.0, print_steps=False, debug=False,
                                                  deviates=deviates, deviate_plot_quantiles=True)

## Zircon Geothermometer

In [ ]:
def zircon_geothermometer(use_composition='rhyolite', # str references dictionary element in composition_d, else dictionary
                          ref_Zr_ppm=100, p=2000.0, tmin=500.0, tmax=1200.0, 
                          print_steps=True, debug=False):
    global composition_d, core, plt, sci, np, rMELTS_ZR
    #
    if isinstance(use_composition, str):
        grm_oxides = composition_d[use_composition]
    else:
        grm_oxides = use_composition
    elm_sys = ['H','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni']
    tot_grm_oxides = 0.0
    for key in grm_oxides.keys():
        tot_grm_oxides += grm_oxides[key]
    mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
    mol_elm = core.chem.mol_oxide_to_elem(mol_oxides)
    elm_ref = np.zeros(107)
    for i,sym in enumerate(
        ['Si', 'Ti', 'Al', 'Fe', 'Cr', 'Mn', 'Mg', 'Ni', 'Co', 'Ca', 'Na', 'K', 'P', 'H', 'C', 'O']):
        index = core.chem.PERIODIC_ORDER.tolist().index(sym)
        elm_ref[index] = mol_elm[i]
    #
    elm = np.copy(elm_ref)
    Zr_ppm = ref_Zr_ppm
    wtZrO2 = 123.218*Zr_ppm/91.224/1000000.0
    wtZrO2 *= tot_grm_oxides + wtZrO2
    elm[core.chem.PERIODIC_ORDER.tolist().index('Zr')] += wtZrO2/123.218
    elm[core.chem.PERIODIC_ORDER.tolist().index('O')]  += wtZrO2*2.0/123.218
    n = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_conv_elm_to_moles(elm)
    if not rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_test_moles(n):
        print ('Output of intrinsic composition calculation fails tests for permissible values.')
        return
    # Function used by minimizer
    def zero(t):
        global rMELTS_ZR
        mu = rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_dgdn(t,p,n)
        y = zirconium_min_affinities(t, p, mu, print_results=False)[0]
        if debug:
            print (t)
        return y
    #
    remember_param_values = rMELTS_ZR.cy_Liquid_rMELTS_ZR_get_param_values()
    results = np.zeros(deviates.shape[0])
    for irow,row in enumerate(deviates): 
        rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(5,  ref_param_values[ 5] + adj_H_ZrSiO4 - 225000.0 + row[0]) # Na4ZrSi2O8
        rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(6,  ref_param_values[ 6] + adj_S_ZrSiO4            + row[3])
        rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(7,  ref_param_values[ 7] + adj_V_ZrSiO4            + row[4])
        rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(8,  ref_param_values[ 8] + adj_H_ZrSiO4 -  75000.0 + row[1]) # K4ZrSi2O8
        rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(9,  ref_param_values[ 9] + adj_S_ZrSiO4            + row[3])
        rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(10, ref_param_values[10] + adj_V_ZrSiO4            + row[4])
        rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(11, ref_param_values[11] + adj_H_ZrSiO4            + row[2]) # ZrSiO4
        rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(12, ref_param_values[12] + adj_S_ZrSiO4            + row[3])
        rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(13, ref_param_values[13] + adj_V_ZrSiO4            + row[4])
        try:
            sol = sci.optimize.root_scalar(zero, bracket=[tmin, tmax])
        except ValueError:
            print ('No valid temperature solution in interval', tmin, 'to', tmax)
            return
        results[irow] = sol.root
    rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_values(remember_param_values)
    t_mean = np.quantile(results, 0.50) - 273.15
    t_05   = np.quantile(results, 0.05) - 273.15
    t_95   = np.quantile(results, 0.95) - 273.15
    return (t_mean, t_05, t_95)
    

In [ ]:
zircon_geothermometer(p=1750)

In [ ]:
zircon_geothermometer(use_composition='dacite', ref_Zr_ppm=150)

In [ ]:
zircon_geothermometer(use_composition = {
    'SiO2':  63.58, 
    'TiO2':   0.64, 
    'Al2O3': 16.67, 
    'Fe2O3':  2.24,
    'Cr2O3':  0.0, 
    'FeO':    3.00, 
    'MnO':    0.11,
    'MgO':    2.12, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    5.53, 
    'Na2O':   3.98, 
    'K2O':    1.40, 
    'P2O5':   0.0, 
    'H2O':    3.00,
    'CO2':    0.05
}, ref_Zr_ppm=200)

Na<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> + 4KAlSiO<sub>4</sub> = K<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> + 2Na<sub>2</sub>SiO<sub>3</sub> + 2Al<sub>2</sub>O<sub>3</sub> + 2SiO<sub>2</sub>

In [ ]:
t = 1000.00
p = 1000.0
def exchange_prop(t=1000, p=1000, param_test=ref_param_values):
    mu0_SiO2    =  rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_endmember_mu0(    0, t, p)
    mu0_Al2O3   =  rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_endmember_mu0(    2, t, p)
    mu0_Na2SiO3 =  rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_endmember_mu0(   11, t, p)
    mu0_KAlSiO4 =  rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_endmember_mu0(   12, t, p)
    S0_SiO2     = -rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_endmember_dmu0dT( 0, t, p)
    S0_Al2O3    = -rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_endmember_dmu0dT( 2, t, p)
    S0_Na2SiO3  = -rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_endmember_dmu0dT(11, t, p)
    S0_KAlSiO4  = -rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_endmember_dmu0dT(12, t, p)
    V0_SiO2     =  rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_endmember_dmu0dP( 0, t, p)
    V0_Al2O3    =  rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_endmember_dmu0dP( 2, t, p)
    V0_Na2SiO3  =  rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_endmember_dmu0dP(11, t, p)
    V0_KAlSiO4  =  rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_endmember_dmu0dP(12, t, p)
    H0_SiO2     = mu0_SiO2    + t*S0_SiO2    - (p-1.0)*V0_SiO2
    H0_Al2O3    = mu0_Al2O3   + t*S0_Al2O3   - (p-1.0)*V0_Al2O3
    H0_Na2SiO3  = mu0_Na2SiO3 + t*S0_Na2SiO3 - (p-1.0)*V0_Na2SiO3
    H0_KAlSiO4  = mu0_KAlSiO4 + t*S0_KAlSiO4 - (p-1.0)*V0_KAlSiO4
    deltaH = param_test[ 8] + 2*H0_Na2SiO3 + 2*H0_Al2O3 + 2*H0_SiO2 - param_test[ 5] - 4*H0_KAlSiO4 
    deltaS = param_test[ 9] + 2*S0_Na2SiO3 + 2*S0_Al2O3 + 2*S0_SiO2 - param_test[ 6] - 4*S0_KAlSiO4 
    deltaV = param_test[10] + 2*V0_Na2SiO3 + 2*V0_Al2O3 + 2*V0_SiO2 - param_test[ 7] - 4*V0_KAlSiO4 
    return (deltaH, deltaS, deltaV)
exchange_prop()

In [ ]:
exchange_prop(param_test=rMELTS_ZR.cy_Liquid_rMELTS_ZR_get_param_values())

## Save revised parameter values

In [ ]:
new_param_values = rMELTS_ZR.cy_Liquid_rMELTS_ZR_get_param_values()
new_param_values

In [ ]:
import pickle

with open('calib_params.pickle', 'wb') as f:
    pickle.dump(new_param_values, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

with open('calib_params.pickle', 'rb') as f:
    params_stored = pickle.load(f)

In [ ]:
params_stored

## Evaluate data from the experimental study of Gervasoni et al. (2016)

In [ ]:
df4 = pd.read_excel('Gervasoni_2016.xlsx')
df4.fillna(0)

In [ ]:
import matplotlib.colors as mcolors
fig = plt.figure(figsize=(15,10))
df_iterator = df4.itertuples()
for row_indx,data in enumerate(df_iterator):
    compo = {
        'SiO2':  data[6], 
        'TiO2':  data[7],
        'Al2O3': data[8], 
        'Fe2O3': data[9]*0.1,
        'Cr2O3': 0.0, 
        'FeO':   data[9]*0.9, 
        'MnO':   0.0,
        'MgO':   data[10], 
        'NiO':   0.0, 
        'CoO':   0.0,
        'CaO':   data[11], 
        'Na2O':  data[12],
        'K2O':   data[13], 
        'P2O5':  0.0, 
        'H2O':   0.1,
        'CO2':   0.01
    }
    result = zircon_geothermometer(use_composition=compo, ref_Zr_ppm=data[2], tmax=2000.0, p=data[5])
    result_l = zircon_geothermometer(use_composition=compo, ref_Zr_ppm=data[2]-data[3], tmax=2000.0, p=data[5])
    result_h = zircon_geothermometer(use_composition=compo, ref_Zr_ppm=data[2]+data[3], tmax=2000.0, p=data[5])
    plt.errorbar(data[2], data[4], xerr=data[3], fmt='ro', label=data[1]+' '+str(data[2])) # mean, low, high
    plt.errorbar(data[2], result[0], 
                 yerr=np.array([[np.fabs(result[1]-result[0])], [np.fabs(result[2]-result[0])]]), fmt='bo')
    plt.errorbar(data[2]-data[3], result_l[0], 
                 yerr=np.array([[np.fabs(result_l[1]-result_l[0])], [np.fabs(result_l[2]-result_l[0])]]), fmt='b,')  
    plt.errorbar(data[2]+data[3], result_h[0], 
                 yerr=np.array([[np.fabs(result_h[1]-result_h[0])], [np.fabs(result_h[2]-result_h[0])]]), fmt='b,')
    plt.plot(np.array([data[2], data[2]]), np.array([data[4], result[0]]), 'k--')
    plt.plot(np.array([data[2]-data[3], data[2]+data[3]]), np.array([result_l[1], result_h[1]]), 'b-')
    plt.plot(np.array([data[2]-data[3], data[2]+data[3]]), np.array([result_l[2], result_h[2]]), 'b-')
    plt.fill(np.array([data[2]-data[3], data[2]+data[3], data[2]+data[3], data[2]-data[3], data[2]-data[3]]),
             np.array([result_l[2], result_h[2], result_h[1], result_l[1], result_l[2]]), 
             color=mcolors.CSS4_COLORS['lightyellow'])
plt.title('Gervasoni et al., 2016, temperature recovery')
plt.ylabel('T °C')
plt.xlabel('Zr (PPM)')
plt.legend()
fig.savefig("Gervasoni_et_al.pdf", bbox_inches='tight')
plt.show()

## Calibrant temperature recovery

In [ ]:
plot_watson = False
plot_boenke = True
fig, main_plt = plt.subplots(figsize=(15,10))
T_meas = []
T_estm = []
T_estm_l = []
T_estm_h = []
lst = []
if plot_watson:
    lst.append(df1)
if plot_boenke:
    lst.append(df2)
for df in lst:
    df_iterator = df.itertuples()
    #
    for row_indx,data in enumerate(df_iterator):
        #print (row_indx)
        auth_id = data[3]
        t = data[25 if auth_id == 1 else 17]
        p = data[26 if auth_id == 1 else 18]*10000.0
        if data[4] != 'Liquid':
            continue
        compo = {
            'SiO2':  data[5 if auth_id == 1 else 5], 
            'TiO2':  data[7 if auth_id == 1 else 6], 
            'Al2O3': data[9 if auth_id == 1 else 7], 
            'Fe2O3': 0.0,
            'Cr2O3': 0.0, 
            'FeO':   data[11 if auth_id == 1 else 8], 
            'MnO':    0.0,
            'MgO':   data[13 if auth_id == 1 else 9], 
            'NiO':    0.0, 
            'CoO':    0.0,
            'CaO':   data[15 if auth_id == 1 else 10], 
            'Na2O':  data[17 if auth_id == 1 else 11], 
            'K2O':   data[19 if auth_id == 1 else 12], 
            'P2O5':   0.0, 
            'H2O':   data[24 if auth_id == 1 else 15],
            'CO2':    0.01
        }
        zr_ppm     = 91.224*1000000.0*data[22 if auth_id == 1 else 13]/100.0/123.218
        zr_ppm_err = 91.224*1000000.0*data[23 if auth_id == 1 else 14]/100.0/123.218
        result = zircon_geothermometer(use_composition=compo, ref_Zr_ppm=zr_ppm, tmax=2000.0, p=p)
        result_l = zircon_geothermometer(use_composition=compo, ref_Zr_ppm=zr_ppm-zr_ppm_err, tmax=2000.0, p=p)
        result_h = zircon_geothermometer(use_composition=compo, ref_Zr_ppm=zr_ppm+zr_ppm_err, tmax=2000.0, p=p)
        T_meas.append(t)
        T_estm.append(result)
        T_estm_l.append(result_l)
        T_estm_h.append(result_h)
        main_plt.errorbar(zr_ppm, t, xerr=zr_ppm_err, fmt='ro')
        main_plt.errorbar(zr_ppm, result[0], 
                 yerr=np.array([[result[0]-result[1]], [result[2]-result[0]]]), fmt='co')
        main_plt.errorbar(zr_ppm-zr_ppm_err, result_l[0], 
                 yerr=np.array([[result_l[0]-result_l[1]], [result_l[2]-result_l[0]]]), fmt='y,')  
        main_plt.errorbar(zr_ppm+zr_ppm_err, result_h[0], 
                 yerr=np.array([[result_h[0]-result_h[1]], [result_h[2]-result_h[0]]]), fmt='y,')
        main_plt.plot(np.array([zr_ppm, zr_ppm]), np.array([t, result[0]]), 'k--')
        main_plt.plot(np.array([zr_ppm-zr_ppm_err, zr_ppm, zr_ppm+zr_ppm_err]), np.array([result_l[1], result[1], result_h[1]]), 'y-')
        main_plt.plot(np.array([zr_ppm-zr_ppm_err, zr_ppm, zr_ppm+zr_ppm_err]), np.array([result_l[2], result[2], result_h[2]]), 'y-')
        main_plt.fill(np.array([zr_ppm-zr_ppm_err, zr_ppm, zr_ppm+zr_ppm_err, zr_ppm+zr_ppm_err, zr_ppm, zr_ppm-zr_ppm_err, zr_ppm-zr_ppm_err]),
             np.array([result_l[2], result[2], result_h[2], result_h[1], result[1], result_l[1], result_l[2]]), 
             color=mcolors.CSS4_COLORS['lightyellow'])
#
inset_plt = fig.add_axes([0.5, 0.2, 0.35, 0.35])
inset_plt.plot(np.array(T_meas), np.array(T_estm)[:,0], 'ro')
inset_plt.errorbar(np.array(T_meas), np.array(T_estm)[:,0], 
                 yerr=[np.array(T_estm)[:,0]-np.array(T_estm)[:,1], np.array(T_estm)[:,2]-np.array(T_estm)[:,0]], fmt='ro')

inset_plt.set_xlabel('T °C, reported')
inset_plt.set_ylabel('T °C, estimated')
#
main_plt.set_ylabel('T °C')
main_plt.set_xlabel('Zr (PPM)')
if plot_watson and not plot_boenke:
    main_plt.set_xlim([0., 4000.])
    main_plt.set_ylim([690.,1100.])
    inset_plt.set_xlim([690.,1100.])
    inset_plt.set_ylim([690.,1100.])
    inset_plt.plot(np.array([690,1100]), np.array([690,1100]), 'k-')
    main_plt.set_title('Watson and Harrison, 1983, temperature recovery')
    fig.savefig("WandH_1983_temperature_recovery.pdf", bbox_inches='tight')
elif not plot_watson and plot_boenke:
    main_plt.set_xlim([0., 4000.])
    main_plt.set_ylim([700.,1100.])
    inset_plt.set_xlim([700.,1100.])
    inset_plt.set_ylim([700.,1100.])
    inset_plt.plot(np.array([700,1100]), np.array([700,1100]), 'k-')
    main_plt.set_title('Boehnke et al., 2013, temperature recovery')
    fig.savefig("Boehnke_2013_temperature_recovery.pdf", bbox_inches='tight')
else:
    inset_plt.plot(np.array([700,1250]), np.array([700,1250]), 'k-')
    main_plt.set_title('Temperature recovery')
    fig.savefig("Temperature_recovery.pdf", bbox_inches='tight')
plt.show()

## Statistics of calibrant temperature recovery

In [ ]:
low   = np.array(T_estm)[:,0] - np.array(T_estm)[:,1]
hgh   = np.array(T_estm)[:,2] - np.array(T_estm)[:,0]
low_l = np.array(T_estm_l)[:,0] - np.array(T_estm_l)[:,1]
hgh_l = np.array(T_estm_l)[:,2] - np.array(T_estm_l)[:,0]
low_h = np.array(T_estm_h)[:,0] - np.array(T_estm_h)[:,1]
hgh_h = np.array(T_estm_h)[:,2] - np.array(T_estm_h)[:,0]
if plot_watson and not plot_boenke:
    pass
elif not plot_watson and plot_boenke:
    low = low[:-4]
    hgh = hgh[:-4]
    low_l = low_l[:-4]
    hgh_l = hgh_l[:-4]
    low_h = low_h[:-4]
    hgh_h = hgh_h[:-4]
print ('5% quartile T (ave Zr):', -np.average(low), '95%', np.average(hgh))
print ('5% quartile T (low Zr):', -np.average(low_l), '95%', np.average(hgh_l))
print ('5% quartile T (hgh Zr):', -np.average(low_h), '95%', np.average(hgh_h))

In [ ]:
err = np.array(T_estm)[:,0] - np.array(T_meas)
err_l = np.array(T_estm_l)[:,0] - np.array(T_meas)
err_h = np.array(T_estm_h)[:,0] - np.array(T_meas)
if plot_watson and not plot_boenke:
    pass
elif not plot_watson and plot_boenke:
    err = err[:-4]
    err_l = err_l[:-4]
    err_h = err_h[:-4]
print ('T est ave (ave Zr) - T meas', np.average(err), '±', np.std(err))
print ('T est ave (low Zr) - T meas', np.average(err_l), '±', np.std(err_l))
print ('T est ave (hgh Zr) - T meas', np.average(err_h), '±', np.std(err_h))